# Roberto Alvarez & Marcos Crespo

We first import the file and create the Pandas' Dataframe:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving wind_available.csv.gzip to wind_available.csv.gzip


In [ ]:
import pandas as pd
df = pd.read_csv('wind_available.csv.gzip', compression="gzip")

# 1. Relevant information and additional documents

Prior to the document developmentm, we wanted to point out that the main original (in the sense that it is not only what its been asked in the document) step we have made is in relation to the dates.

He have combined all the date-related features of our data into one date-datatype column, which exists in pandas. This way, we could made a clear cut in the time-series of our data in order to make the predictions for a non iid data. Since we were not asked to make any time-series model, we the deleted this feature and worked only with the timeless data.

The main reason we had to do this was that the competition data we observed was in 2010, and our data only covered 2009. The standard procedure when doing this is to create the train-test partitions with a time-threshold, not shuffling the data.

Please find attached in the .zip file the aditional document required for this assigment, commenting the main use cases we have found to ChatGPT.

## 2. EDA

First step is to create a simple Exploratory Data Analysis in order to have a brief understanding of quality, quantity and general characteristics of the data.

In [ ]:

# Checking the number of instances (rows) and attributes (columns)
num_instances, num_attributes = df.shape
print("Number of instances (rows):", num_instances)
print("Number of attributes (columns):", num_attributes)

# Display data type information for each column
print(df.dtypes)


Number of instances (rows): 4748
Number of attributes (columns): 555
energy     float64
year         int64
month        int64
day          int64
hour         int64
            ...   
v100.21    float64
v100.22    float64
v100.23    float64
v100.24    float64
v100.25    float64
Length: 555, dtype: object


Typically, columns with data type object or category represent categorical variables, while columns with data types such as int64 or float64 represent numerical variables. We have 4748 instances (rows) and 555 attributes (columns) in our dataset. The 'energy' column, our target variable, it's of type 'float64' this represents the output in a regression problem. Other columns, such as 'year', 'month', 'day', 'hour', and several others, seem to contain numerical data as well, represented by 'int64' or 'float64' data types.

No categorical or character columns are observed in this data.

In [ ]:
## Missing values

#Calculate the percentage of missing values for each column
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Sort columns by their missing percentage in descending order
missing_percentage = missing_percentage.sort_values(ascending=False)

# Display the result
print(missing_percentage)


lai_hv.13    20.303286
fsr.3        20.155855
v10n.24      19.945240
stl3.20      19.924179
lai_hv.19    19.839933
               ...    
hour          0.000000
year          0.000000
day           0.000000
month         0.000000
energy        0.000000
Length: 555, dtype: float64


We have several variables with missing values, to handle it we will impute them with the median

In [ ]:
#Imputation
# Select numerical columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Impute missing values with the median for numerical columns
for column in numeric_columns:
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)


In [ ]:
# Check for missing values after imputation
missing_after_imputation = df.isnull().sum()
print("Missing values after imputation:")
print(missing_after_imputation)


Missing values after imputation:
energy     0
year       0
month      0
day        0
hour       0
          ..
v100.21    0
v100.22    0
v100.23    0
v100.24    0
v100.25    0
Length: 555, dtype: int64


Now we dont have missing values

In [ ]:
# Check for constant columns
constant_columns = [col for col in df.columns if df[col].nunique() == 1]

# Display constant columns
print("Constant columns:")
print(constant_columns)


Constant columns:
[]


In this dataset there are not constant columns

# 3. Split Data

In [ ]:
# Check unique values and their counts in the 'year' column
year_counts = df['year'].value_counts().sort_index()

# Display the unique values and their counts
print("Unique values and their counts in the 'year' column:")
print(year_counts)


Unique values and their counts in the 'year' column:
2005    1256
2006    1272
2007    1121
2008     178
2009     921
Name: year, dtype: int64


Data is not iid, it follows a temporal order, we will bear that in mind to split the data, since our goal is to forecast future energy production, the test set will contain data from the most recent time periods to simulate real-world predictions.

In [ ]:
# 'df' has columns 'year', 'month', 'day', and 'hour'
# Make sure the columns have the correct type (integer) before combining them

df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# Now, you can delete the original columns if you wish
df = df.drop(['year', 'month', 'day', 'hour'], axis=1)

# Visualize the resulting DataFrame
print(df.head())



    energy     p54.162.1     p54.162.2     p54.162.3     p54.162.4  \
0   402.71  2.534970e+06  2.526864e+06  2.518754e+06  2.510648e+06   
1   696.80  2.513508e+06  2.505370e+06  2.521184e+06  2.513088e+06   
2  1591.15  2.533727e+06  2.525703e+06  2.517678e+06  2.509654e+06   
3  1338.62  2.513508e+06  2.526548e+06  2.518609e+06  2.510670e+06   
4   562.50  2.529543e+06  2.505370e+06  2.513702e+06  2.505782e+06   

      p54.162.5     p54.162.6     p54.162.7     p54.162.8     p54.162.9  ...  \
0  2.502537e+06  2.531111e+06  2.522721e+06  2.514330e+06  2.505940e+06  ...   
1  2.482535e+06  2.533465e+06  2.525088e+06  2.516716e+06  2.508339e+06  ...   
2  2.482535e+06  2.529801e+06  2.521496e+06  2.513187e+06  2.504882e+06  ...   
3  2.502732e+06  2.530569e+06  2.502431e+06  2.514127e+06  2.505904e+06  ...   
4  2.497861e+06  2.525621e+06  2.517421e+06  2.509215e+06  2.501015e+06  ...   

    v100.17   v100.18   v100.19   v100.20   v100.21   v100.22   v100.23  \
0 -0.450284 -4.407196 -

In [ ]:
# 'X' contains features and 'y' contains the target variable
X = df.drop(columns=['energy'])  # Features
y = df['energy']  # Target variable

# Define the cutoff year for the test set (e.g., most recent year)
cutoff = pd.to_datetime('2009-01-01 00:00')

# Filter data based on 'year' for training and test sets
X_train = X[df['datetime'] < cutoff]
y_train = y[df['datetime'] < cutoff]
X_test = X[df['datetime'] >= cutoff]
y_test = y[df['datetime'] >= cutoff]

# Verify the shapes or time periods covered in the sets
print("Training set period:", X_train['datetime'].min(), "-", X_train['datetime'].max())  # Just for verification
print("Test set period:", X_test['datetime'].min(), "-", X_test['datetime'].max())  # Just for verification
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

# Drop the 'datetime' column from X_train and X_test
X_train = X_train.drop(columns=['datetime'])
X_test = X_test.drop(columns=['datetime'])


Training set period: 2005-01-02 18:00:00 - 2008-02-25 18:00:00
Test set period: 2009-03-05 12:00:00 - 2009-12-31 18:00:00
Train set shape: (3827, 551) (3827,)
Test set shape: (921, 551) (921,)


For this regression problem (energy prediction), metrics like Mean Absolute Error (MAE) or Mean Squared Error (MSE) can be suitable for assessing model performance

# 4. Default Hyper Parameters

Using default hyper-parameters, we evaluate Trees and KNN on the testing partition.





*Decision Trees*

In [ ]:
from sklearn import tree
from sklearn import metrics

# Initialize Decision Tree Regressor
tree_regressor = tree.DecisionTreeRegressor(random_state=0)

# Fit the model on the training data
tree_regressor.fit(X_train, y_train)

# Predict on the test set
tree_predictions = tree_regressor.predict(X_test)

# Evaluate Decision Tree performance using Mean Squared Error (MSE)
tree_mse = (metrics.mean_squared_error(y_test, tree_predictions))
print("Decision Tree MSE:", tree_mse)


Decision Tree MSE: 279096.0921464712


*KNN*

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Initialize KNN model
knn_model = KNeighborsRegressor()

# Create pipelines for scaling methods (StandardScaler and MinMaxScaler)
scalers = [('StandardScaler', StandardScaler()), ('MinMaxScaler', MinMaxScaler())]

# Evaluate KNN with different scaling methods
for scaler_name, scaler in scalers:
    # Create pipeline with scaler and KNN model
    knn_pipeline = Pipeline([('scaler', scaler), ('knn', knn_model)])

    # Fit the pipeline on the training data
    knn_pipeline.fit(X_train, y_train)

    # Predict on the test set
    knn_predictions = knn_pipeline.predict(X_test)

    # Evaluate KNN performance (e.g., using Mean Squared Error)
    knn_mse = mean_squared_error(y_test, knn_predictions)
    print(f"KNN with {scaler_name} MSE:", knn_mse)


KNN with StandardScaler MSE: 204958.38570563303
KNN with MinMaxScaler MSE: 233285.20356867753


# 5. HPO

Now, we do hyper-parameter tuning (HPO) for trees and KNN, for the search method we use the Grid and Random search

*Grid search for trees*

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Define hyperparameters to tune for Decision Tree
tree_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize Grid Search with the Decision Tree model and parameter grid
tree_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=0), tree_param_grid, cv=5, scoring='neg_mean_squared_error')
tree_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_tree_params = tree_grid_search.best_params_
best_tree_mse = -tree_grid_search.best_score_

print("Best Decision Tree Hyperparameters:", best_tree_params)
print("Best Decision Tree MSE:", best_tree_mse)


# Predict on the test set using the best tree model obtained from grid search
best_tree_model = tree_grid_search.best_estimator_
y_pred_test = best_tree_model.predict(X_test)

# Calculate the mean squared error on the test set
test_mse = mean_squared_error(y_test, y_pred_test)
print("Decision Tree MSE on Test Set:", test_mse)


Best Decision Tree Hyperparameters: {'max_depth': 10, 'min_samples_split': 10}
Best Decision Tree MSE: 220582.52402876574
Decision Tree MSE on Test Set: 237842.5910418732


*Grid Search for KNN*

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Create KNN model
knn_model = KNeighborsRegressor()

# Create pipeline with scaler and KNN model for grid search
knn_pipeline = Pipeline([('scaler', StandardScaler()), ('knn', knn_model)])

# Define hyperparameters to tune for KNN within the pipeline
knn_param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()],  # Test different scalers
    'knn__n_neighbors': range(2,16,2),  # Example hyperparameter for KNN
    # Other hyperparameters for KNN...
}

# Initialize Grid Search with the KNN pipeline and parameter grid
knn_grid_search = GridSearchCV(knn_pipeline, knn_param_grid, cv=5, scoring='neg_mean_squared_error')
knn_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_knn_params = knn_grid_search.best_params_
best_knn_mse = -knn_grid_search.best_score_

print("Best KNN Hyperparameters:", best_knn_params)
print("Best KNN MSE:", best_knn_mse)



# Predict on the test set using the best KNN model obtained from grid search
best_knn_model = knn_grid_search.best_estimator_
y_pred_test_knn = best_knn_model.predict(X_test)

# Calculate the mean squared error on the test set
test_mse_knn = mean_squared_error(y_test, y_pred_test_knn)
print("KNN MSE on Test Set:", test_mse_knn)


Best KNN Hyperparameters: {'knn__n_neighbors': 14, 'scaler': StandardScaler()}
Best KNN MSE: 188706.2455534932
KNN MSE on Test Set: 196662.78416797734


*Random Search for Trees*

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import randint

# Define hyperparameters to tune for Decision Tree
tree_param_dist = {
    'max_depth': randint(1, 31),  # Example range for max_depth using sp_randint
    'min_samples_split': randint(2, 11)  # Example range for min_samples_split using sp_randint
    # Add other hyperparameters for Decision Tree here...
}

# Initialize Randomized Search with the Decision Tree model and parameter distribution
tree_random_search = RandomizedSearchCV(DecisionTreeRegressor(random_state=0), tree_param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
tree_random_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_tree_params = tree_random_search.best_params_
best_tree_mse = -tree_random_search.best_score_

print("Best Decision Tree Hyperparameters:", best_tree_params)
print("Best Decision Tree MSE:", best_tree_mse)

# Predict on the test set using the best Decision Tree model obtained from random search
best_tree_model = tree_random_search.best_estimator_
y_pred_test_tree = best_tree_model.predict(X_test)

# Calculate the mean squared error on the test set
test_mse_tree = mean_squared_error(y_test, y_pred_test_tree)
print("Decision Tree MSE on Test Set:", test_mse_tree)


Best Decision Tree Hyperparameters: {'max_depth': 7, 'min_samples_split': 4}
Best Decision Tree MSE: 198685.40847071083
Decision Tree MSE on Test Set: 219156.9537641815


*Random Search for KNN*

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import randint

# Create KNN model
knn_model = KNeighborsRegressor()

# Create pipeline with scaler and KNN model for random search
knn_pipeline = Pipeline([('scaler', StandardScaler()), ('knn', knn_model)])

# Define hyperparameters to tune for KNN within the pipeline
knn_param_dist = {
    'scaler': [StandardScaler(), MinMaxScaler()],  # Test different scalers
    'knn__n_neighbors': randint(2, 16)  # Example hyperparameter for KNN
    # Other hyperparameters for KNN...
}

# Initialize Randomized Search with the KNN pipeline and parameter distribution
knn_random_search = RandomizedSearchCV(knn_pipeline, knn_param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
knn_random_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_knn_params = knn_random_search.best_params_
best_knn_mse = -knn_random_search.best_score_

print("Best KNN Hyperparameters:", best_knn_params)
print("Best KNN MSE:", best_knn_mse)

# Predict on the test set using the best KNN model obtained from random search
best_knn_model = knn_random_search.best_estimator_
y_pred_test_knn = best_knn_model.predict(X_test)

# Calculate the mean squared error on the test set
test_mse_knn = mean_squared_error(y_test, y_pred_test_knn)
print("KNN MSE on Test Set:", test_mse_knn)


Best KNN Hyperparameters: {'knn__n_neighbors': 14, 'scaler': StandardScaler()}
Best KNN MSE: 188706.2455534932
KNN MSE on Test Set: 196662.78416797734


The summary of the models so far is:


|                 | Decision Trees | KNN |
|-----------------|----------------|-----|
| **Default**     | 279096.09 | 204958.38|
| **Grid Search** |  237842.59  | 196662.78 |
| **Random Search**|     219156.95 |  196662.78 |

The best model so far is a KNN with 14 neighbors and StandarScaler, obtained twice from both the Grid and Random search methods.



**FINE TUNING KNN**

In addition to the models proposed in the assigment, we will be trying to perform a Fine Tunning KNN, as a further step in the HPO for the best model we obtained, this is, the StandardScaler KNN.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Define hyperparameters to fine-tune for KNN
knn_param_grid_fine_tune = {
    'scaler': [StandardScaler()],
    'knn__n_neighbors': range(12, 18),  # Narrow the range around the best value
    'knn__weights': ['uniform', 'distance'],  # Try different weighting schemes
    'knn__p': [1, 2]  # Consider using different distance measures
}

# Initialize Grid Search for finer hyperparameter tuning
knn_grid_search_fine_tune = GridSearchCV(
    knn_pipeline,
    knn_param_grid_fine_tune,
    cv=5,
    scoring='neg_mean_squared_error'
)
knn_grid_search_fine_tune.fit(X_train, y_train)

# Get the best hyperparameters and corresponding MSE
best_knn_params_fine_tune = knn_grid_search_fine_tune.best_params_
best_knn_mse_fine_tune = -knn_grid_search_fine_tune.best_score_

print("Best KNN Hyperparameters (Fine-tuned):", best_knn_params_fine_tune)
print("Best KNN MSE (Fine-tuned):", best_knn_mse_fine_tune)

# Predict on the test set using the best KNN model from fine-tuned search
best_knn_model_fine_tune = knn_grid_search_fine_tune.best_estimator_
y_pred_test_knn_fine_tune = best_knn_model_fine_tune.predict(X_test)

# Calculate the mean squared error on the test set
test_mse_knn_fine_tune = mean_squared_error(y_test, y_pred_test_knn_fine_tune)
print("KNN MSE on Test Set (Fine-tuned):", test_mse_knn_fine_tune)


Best KNN Hyperparameters (Fine-tuned): {'knn__n_neighbors': 16, 'knn__p': 1, 'knn__weights': 'distance', 'scaler': StandardScaler()}
Best KNN MSE (Fine-tuned): 173467.7712369944
KNN MSE on Test Set (Fine-tuned): 177394.30447103587


We see how the MSE improves in the fine-tunned model by more than 10%, so the additional computational cost pays off.

# 6. Using the best method of the ones evaluated previously:

1.   Final model: using the best method, train the final model and use it to make predictions on the competition dataset. Save both the final model and the competition predictions on files.


We first upload the competition data the same way we uploaded the give data.

In [ ]:
uploaded_comp = files.upload()

Saving wind_competition.csv.gzip to wind_competition.csv.gzip


In [ ]:
df_comp = pd.read_csv('wind_competition.csv.gzip', compression="gzip")

And now, the same preprocessing done to the train and test sets
must be performed.

In [ ]:
## Missing values

#Calculate the percentage of missing values for each column
missing_percentage_comp = (df_comp.isnull().sum() / len(df)) * 100

# Sort columns by their missing percentage in descending order
missing_percentage_comp = missing_percentage_comp.sort_values(ascending=False)

# Display the result
print(missing_percentage_comp)



stl3.11    5.412805
fsr.22     5.370682
stl3.1     5.328559
v10n.10    5.286436
u10n.7     5.223252
             ...   
iews.2     1.074136
month      0.000000
hour       0.000000
day        0.000000
year       0.000000
Length: 554, dtype: float64


In [ ]:
#Imputation
# Select numerical columns
numeric_columns = df_comp.select_dtypes(include=['float64', 'int64']).columns

# Impute missing values with the median for numerical columns
for column in numeric_columns:
    median_value = df_comp[column].median()
    df_comp[column].fillna(median_value, inplace=True)


In [ ]:
# Check for missing values after imputation
missing_after_imputation = df_comp.isnull().sum()
print("Missing values after imputation:")
print(missing_after_imputation)


Missing values after imputation:
year         0
month        0
day          0
hour         0
p54.162.1    0
            ..
v100.21      0
v100.22      0
v100.23      0
v100.24      0
v100.25      0
Length: 554, dtype: int64


In [ ]:
# Drop unused variables
df_comp = df_comp.drop(['year', 'month', 'day', 'hour'], axis=1)

print(df_comp.head())


      p54.162.1     p54.162.2     p54.162.3     p54.162.4     p54.162.5  \
0  2.403131e+06  2.395445e+06  2.387755e+06  2.380065e+06  2.372380e+06   
1  2.410306e+06  2.402394e+06  2.480434e+06  2.386571e+06  2.378660e+06   
2  2.434908e+06  2.426793e+06  2.418683e+06  2.410573e+06  2.402462e+06   
3  2.447112e+06  2.487275e+06  2.431027e+06  2.422984e+06  2.414942e+06   
4  2.459695e+06  2.487275e+06  2.443809e+06  2.435866e+06  2.465786e+06   

      p54.162.6     p54.162.7     p54.162.8     p54.162.9    p54.162.10  ...  \
0  2.399548e+06  2.391582e+06  2.383621e+06  2.375660e+06  2.367699e+06  ...   
1  2.493774e+06  2.398599e+06  2.477355e+06  2.382225e+06  2.374038e+06  ...   
2  2.431465e+06  2.423075e+06  2.414689e+06  2.469211e+06  2.397912e+06  ...   
3  2.443696e+06  2.435378e+06  2.427060e+06  2.418742e+06  2.410423e+06  ...   
4  2.456252e+06  2.448034e+06  2.439815e+06  2.431596e+06  2.423377e+06  ...   

    v100.16   v100.17   v100.18   v100.19   v100.20   v100.21   v100

In [ ]:
# Check for constant columns
constant_columns = [col for col in df_comp.columns if df_comp[col].nunique() == 1]

# Display constant columns
print("Constant columns:")
print(constant_columns)


Constant columns:
[]


Lets make the predictions on the competition data using the best method and save the model and the results

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from joblib import dump
import pandas as pd

#drop datetime fromm X
X = X.drop(columns=['datetime'])

# Define the best hyperparameters obtained from fine-tuning
best_knn_params = {
    'n_neighbors': 16,
    'p': 1,
    'weights': 'distance'
}

# Create the final model with the best parameters
final_model = KNeighborsRegressor(**best_knn_params)

# Apply StandardScaler to the entire available dataset
scaler = StandardScaler()
X_available_scaled = scaler.fit_transform(X)

# Train the final KNN model on the entire available dataset (train + validation sets)
final_model.fit(X_available_scaled, y)

# Save the final KNN model to a file
dump(final_model, 'final_knn_model_fine_tuned.joblib')

# Make predictions on the competition dataset using the trained model and scaler
X_competition_scaled = scaler.transform(df_comp)  # Scale the competition dataset
competition_predictions_knn = final_model.predict(X_competition_scaled)

# Save the competition predictions to a CSV file
competition_predictions_df_knn = pd.DataFrame({'Predictions': competition_predictions_knn})
competition_predictions_df_knn.to_csv('competition_predictions_knn_fine_tuned.csv', index=False)


To download files

In [ ]:
from google.colab import files

# Download the model file
files.download('final_knn_model_fine_tuned.joblib')

# Download the predictions file
files.download('competition_predictions_knn_fine_tuned.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 7. Feature selection for KNN:

Now the goal is to take the model given by the HPO for KNN and find the most important features in this model. For this, we will be using the existing model with 14 Neighbors and Standard Scaler and try to find the subset of best features with Grid Search.

The criteria used are: f_regression and mutual_info_regression.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

# Define the range of features (k) to test in grid search
k_range = [5, 10, 15, 20, 25]
feature_names=df.columns.tolist()

# Initialize KNN model with the optimal number of neighbors
knn_model_with_optimal_neighbors = KNeighborsRegressor(n_neighbors=best_knn_params['knn__n_neighbors'])

# Create a new pipeline with feature selection and KNN model
feature_selection_knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(score_func=f_regression)),  # Use f_regression as the initial criterion
    ('knn', knn_model_with_optimal_neighbors)
])

# Define hyperparameters to tune for feature selection and KNN
feature_selection_param_grid = {
    'feature_selection__k': k_range,  # Test different values of k (number of features)
    'feature_selection__score_func': [f_regression, mutual_info_regression]  # Test different feature selection criteria
    # Other hyperparameters for KNN...
}

# Initialize Grid Search with the feature selection and KNN pipeline and parameter grid
feature_selection_grid_search = GridSearchCV(feature_selection_knn_pipeline, feature_selection_param_grid, cv=5, scoring='neg_mean_squared_error')
feature_selection_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_feature_selection_params = feature_selection_grid_search.best_params_
best_feature_selection_mse = -feature_selection_grid_search.best_score_

print("Best Feature Selection Hyperparameters:", best_feature_selection_params)
print("Best Feature Selection MSE:", best_feature_selection_mse)

# Get the optimal number of features and the corresponding selected features
optimal_num_features = best_feature_selection_params['feature_selection__k']
selected_features = feature_selection_grid_search.best_estimator_.named_steps['feature_selection'].get_support()

# Print the results
print("Optimal Number of Features:", optimal_num_features)
selected_feature_names = [feature_names[i] for i, selected in enumerate(selected_features) if selected]
print("Selected Feature Names:", selected_feature_names)


Best Feature Selection Hyperparameters: {'feature_selection__k': 5, 'feature_selection__score_func': <function mutual_info_regression at 0x7a3cf9da16c0>}
Best Feature Selection MSE: 197038.5706384718
Optimal Number of Features: 5
Selected Feature Names: ['iews.5', 'iews.12', 'iews.13', 'iews.17', 'u100.10']


**Feature selection with Fine tuning model**

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

# Define the range of features (k) to test in feature selection
k_range = [5, 10, 15, 20, 25]
feature_names = df.columns.tolist()

# Initialize KNN model with the best hyperparameters
knn_model_fine_tuned = KNeighborsRegressor(n_neighbors=16, p=1, weights='distance')

# Create a new pipeline with feature selection and KNN model
feature_selection_knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(score_func=f_regression)),
    ('knn', knn_model_fine_tuned)
])

# Define hyperparameters to tune for feature selection
feature_selection_param_grid = {
    'feature_selection__k': k_range,
    'feature_selection__score_func': [f_regression, mutual_info_regression]
    # Other hyperparameters for KNN can be added here if necessary
}

# Initialize Grid Search with the feature selection and KNN pipeline and parameter grid
feature_selection_grid_search = GridSearchCV(
    feature_selection_knn_pipeline,
    feature_selection_param_grid,
    cv=5,
    scoring='neg_mean_squared_error'
)
feature_selection_grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding MSE
best_feature_selection_params = feature_selection_grid_search.best_params_
best_feature_selection_mse = -feature_selection_grid_search.best_score_

print("Best Feature Selection Hyperparameters:", best_feature_selection_params)
print("Best Feature Selection MSE:", best_feature_selection_mse)

# Get the optimal number of features and the corresponding selected features
optimal_num_features = best_feature_selection_params['feature_selection__k']
selected_features = feature_selection_grid_search.best_estimator_.named_steps['feature_selection'].get_support()

# Print the results
print("Optimal Number of Features:", optimal_num_features)
selected_feature_names = [feature_names[i] for i, selected in enumerate(selected_features) if selected]
print("Selected Feature Names:", selected_feature_names)


Best Feature Selection Hyperparameters: {'feature_selection__k': 25, 'feature_selection__score_func': <function mutual_info_regression at 0x7dde07279360>}
Best Feature Selection MSE: 191673.4369844064
Optimal Number of Features: 25
Selected Feature Names: ['iews.1', 'iews.3', 'iews.5', 'iews.7', 'iews.9', 'iews.10', 'iews.11', 'iews.12', 'iews.13', 'iews.16', 'iews.17', 'iews.18', 'iews.19', 'iews.20', 'iews.22', 'iews.23', 'iews.24', 'u100.3', 'u100.6', 'u100.7', 'u100.10', 'u100.12', 'u100.14', 'u100.21', 'u100.23']
